## HW optional: Word Embeddings

*based on [YSDA](https://github.com/yandexdataschool/nlp_course) materials*

Today we gonna play with word embeddings.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__ if you're running locally, in the selected environment run the following command:

```pip install --upgrade nltk gensim bokeh umap-learn```


In [1]:
import itertools
import string

import numpy as np
from nltk.tokenize import WordPunctTokenizer

from matplotlib import pyplot as plt

from IPython.display import clear_output

In [2]:
# download the data:
!wget https://yadi.sk/i/BPQrUu1NaTduEw -O ./quora.txt -nc

zsh:1: command not found: wget


In [3]:
data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

### Tokenization (1pt)

a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [4]:
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [5]:
# TASK: lowercase everything and extract tokens with tokenizer.
# data_tok should be a list of lists of tokens for each line in data.

data_tok = list(map(lambda x: tokenizer.tokenize(x.lower()), data))

Let's peek at the result:

In [6]:
' '.join(data_tok[0])

"can i get back with my ex even though she is pregnant with another guy ' s baby ?"

Small check that everything is alright

In [7]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

### Word2Vec (1 pt)

 as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another NLP library that features many vector-based models incuding word2vec.

In [8]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok,
                 vector_size=32,      # embedding vector size
                 min_count=5,      # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [9]:
# now you can get word vectors !
model.get_vector('anything')

array([-3.644425  ,  1.7465744 ,  0.34989578,  3.3626387 ,  2.3115427 ,
        1.6441923 ,  1.0897188 , -5.212344  ,  0.21016406,  2.8561206 ,
       -0.86907643,  2.1799152 ,  3.1775055 , -0.32997698,  2.9344149 ,
       -1.2039163 , -0.7194742 , -1.84247   ,  0.2502068 , -0.78926194,
       -1.520774  , -0.07615144, -1.3925198 , -2.9789617 ,  2.2102933 ,
       -1.8391595 ,  0.78063256,  1.7367785 ,  1.0744443 ,  0.64179677,
        0.49154547,  1.1030581 ], dtype=float32)

In [10]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

[('rice', 0.9624271988868713),
 ('sauce', 0.9436677694320679),
 ('butter', 0.936170220375061),
 ('vodka', 0.9357630014419556),
 ('beans', 0.9308580160140991),
 ('cheese', 0.9271355867385864),
 ('potato', 0.9224681854248047),
 ('chocolate', 0.9203252196311951),
 ('fruit', 0.9200490117073059),
 ('corn', 0.9131788611412048)]

Show the linear structure in embeddings space with 2-3 examples

In [11]:
model.most_similar(positive=['woman', 'king'], negative=['man']) # expecting queen

[('elizabeth', 0.8704655170440674),
 ('queen', 0.8641855716705322),
 ('lee', 0.8629263639450073),
 ('prince', 0.852715253829956),
 ('abraham', 0.8510167002677917),
 ('jackson', 0.8453159332275391),
 ('stark', 0.8346445560455322),
 ('martin', 0.8298641443252563),
 ('mary', 0.826865553855896),
 ('luther', 0.8218756318092346)]

In [12]:
model.most_similar(positive=['people', 'science'])

[('teachers', 0.748420774936676),
 ('scientists', 0.6994851231575012),
 ('iitians', 0.6896190047264099),
 ('students', 0.6825103163719177),
 ('others', 0.6817463040351868),
 ('discussions', 0.6771730780601501),
 ('professors', 0.6759803891181946),
 ('indians', 0.6732125282287598),
 ('experts', 0.6685829758644104),
 ('mathematicians', 0.66591876745224)]

In [13]:
model.most_similar(positive=['fly', 'vehicle', 'air'], negative=['wheels'])

[('aircraft', 0.7797800302505493),
 ('flight', 0.7552220821380615),
 ('carrier', 0.7517803907394409),
 ('passenger', 0.7418912053108215),
 ('land', 0.7282929420471191),
 ('license', 0.7054904103279114),
 ('operate', 0.7026874423027039),
 ('tank', 0.7021215558052063),
 ('plane', 0.6935926079750061),
 ('wire', 0.6750209927558899)]

### Glove + PCA (1 pt)

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [14]:
import gensim.downloader as api

model = api.load('glove-twitter-25')

In [15]:
words = sorted(model.key_to_index, 
               key=lambda word_key: model.get_vecattr(word_key, "count"),
               reverse=True)[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [16]:
# for each word, compute it's vector with model
word_vectors = np.array([model[word] for word in words])

In [17]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 25)
assert np.isfinite(word_vectors).all()

In [18]:
word_vectors.shape

(1000, 25)

Use PCA to decrease embeddings dimension so that we will be able to draw them.

In [19]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
pca = PCA(2)
scaler = StandardScaler()
# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
word_vectors_pca = pca.fit_transform(word_vectors)
word_vectors_pca = scaler.fit_transform(word_vectors_pca)

In [20]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

Function to draw embeddings

In [21]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [22]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

figure(id='p1004', ...)

<font color=5e60ce>We could see a big English, Emoji, Spanish, Arabic and Indonesian language clusters. </font>

### Phrase Embeddings (1 pt)

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.


In [23]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros

    phrase = phrase.lower()
    tokens = tokenizer.tokenize(phrase)
    vector = np.zeros(word_vectors.shape[1])
    
    cnt = 0
    for token in tokens:
        if token in model.key_to_index:
            vector += model[token]
            cnt += 1
    return vector / cnt if cnt > 0 else vector

In [24]:
get_phrase_embedding(data[402687])

array([-0.18205   ,  0.30953575,  0.20861092,  0.07982158, -0.22565517,
       -0.3300175 ,  1.24957835,  0.13134292, -0.33788875,  0.06196944,
       -0.231793  ,  0.09389218, -4.96854999, -0.23611419, -0.32609667,
       -0.092073  ,  0.4407505 , -0.7541375 , -0.5389092 , -0.184752  ,
        0.07867809,  0.20018657, -0.16202375,  0.303757  , -0.41255667])

In [25]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

In [37]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases and turn them to numpy array
phrase_vectors = np.asarray([get_phrase_embedding(x) for x in chosen_phrases]) # YOUR CODE

In [38]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [44]:
phrase_vectors[0]

array([-0.20129163,  0.28008468,  0.20248146, -0.06290309, -0.13497116,
        0.00861136,  1.07697579,  0.28034084, -0.48734121,  0.22369644,
       -0.21113858,  0.33287615, -4.84304211, -0.35945611, -0.11689068,
        0.06346284,  0.25226158, -0.48065658, -0.50974858, -0.159933  ,
        0.17304395,  0.26804752,  0.10165837,  0.13623684, -0.20279761])

In [55]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize
from sklearn.manifold import TSNE

tsne = TSNE(perplexity=80)
pca_many_dims = PCA(n_components=10)
phrase_vectors_2d = pca_many_dims.fit_transform(phrase_vectors)
phrase_vectors_2d = tsne.fit_transform(phrase_vectors_2d)
phrase_vectors_2d = scaler.fit_transform(phrase_vectors_2d)

In [56]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

figure(id='p1151', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [57]:
# filter all sentences that include at least one NONPRINTABLE char
# compute vector embedding for all remaining lines in data

printable_set = set(string.printable)

data_subset = [x for x in data if set(x).issubset(printable_set)]

In [58]:
data_vectors = np.vstack([get_phrase_embedding(x) for x in data])

norms = np.linalg.norm(data_vectors, axis=1)
norms[norms == 0] = 0.00001

In [59]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    assert len(norms)==len(data_vectors)
    
    query_emb = get_phrase_embedding(query)
    query_norm = np.linalg.norm(query_emb)
    
    cosines = []
    for i in range(len(norms)):
        cosines.append(np.dot(query_emb, data_vectors[i]) / norms[i] / query_norm)
    args = np.flip(np.argsort(cosines))[:k]
    nearest = [data[args[i]] for i in range(10)]
    return nearest

In [60]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'

How do I get to the dark web?
What universal remote do I need and how do I set it up to a Blaupunkt TV?
How do I connect the ASUS_T00Q to my PC?
How do you print the gridlines in Excel 2010?
How do you print the gridlines in Excel 2007?
How do you print the gridlines in Excel 2003?
I would like to create a new website. What do I have to do?
How do I get the new Neko Atsume wallpapers? How do they work?
I want to experience the 4G network. Do I need to change my SIM card from 3G to 4G?
What do I have to do to sell my photography?



In [61]:
find_nearest(query="How does Trump?", k=10)

['What does Donald Trump think about Israel?\n',
 'Who or what is Donald Trump, really?\n',
 'Donald Trump: Why are there are so many questions about Donald Trump on Quora?\n',
 'Does anyone like Trump and Clinton?\n',
 'What does Cortana mean?\n',
 'Did Bill Gates outcompete and outsmart IBM? Why? How?\n',
 'Why and how is Bill Gates so rich?\n',
 'What does Donald Trump think about Pakistan?\n',
 'What do you think about Trump and Obama?\n',
 'Who and what is Quora?\n']

In [62]:
find_nearest(query="Why don't i ask a question myself?", k=10)

["Why don't my parents listen to me?\n",
 "Why don't people appreciate me?\n",
 "Why she don't interact with me?\n",
 "Why don't I get a date?\n",
 "Why don't I get a girlfriend?\n",
 "Why don't I have a girlfriend?\n",
 "Why don't I have a boyfriend?\n",
 "Why don't I like people touching me?\n",
 "Why can't I ask a question anonymously?\n",
 "Why don't you use Facebook much?\n"]

### Phrase Embeddings with TfIdf (2 pt, bonus)

Improve the above __get_phrase_embedding__ function to use TF-IDF as vector weights.

1) train sklearn tf-idf implementation
2) update function *get_phrase_embedding*
3) compare new and old versions using *find_nearest* on 2-3 examples 

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [73]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data)

In [74]:
tfidf_matrix.shape

(537272, 86108)

In [78]:
vectorizer.get_feature_names_out()[10000:10005]

array(['baherain', 'bahia', 'bahira', 'bahr', 'bahrain'], dtype=object)

In [84]:
vectorizer.idf_[100:105]

array([13.50111444, 11.01620779, 13.09564934, 13.50111444, 13.50111444])

### FastText (2 pt, bonus)

Implement search enginge using [FastText](https://fasttext.cc/docs/en/python-module.html) embeddings. 

1) train fasttext embeddings on all data
2) update *get_phrase_embeddings* function
3) implement *find_nearest* function with fasttext embeddings
4) compare results with glove version from the main task

In [ ]:
# YOUR CODE HERE